Inference Notebook for FBA dataset
- Model path: "sdxl-base-1.0-fbadataset5e-4-lrwrmp0-ep20-withoutpadding-noflip-lora-newlabels"
- Training Dataset: "artisanalwasp/resized_fba_with_lanczos_wo_wearscores_refactoredlabels" 

- Install dependencies

In [1]:
! pip install -U peft transformers diffusers

  Attempting uninstall: peft
    Found existing installation: peft 0.7.0
    Uninstalling peft-0.7.0:
      Successfully uninstalled peft-0.7.0


Diffusers Pipeline Setup        
Loading Stable Diffusion XL Base 1.0 as base model then attaching LoRa adapters


In [2]:
%load_ext autoreload
%autoreload 2

from diffusers import DiffusionPipeline
import torch
import os
from src.prompt_generator import PromptGenerator

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

# Optional CPU offloading to save same GPU memory
pipe.enable_model_cpu_offload()

model_path = "artisanalwasp/sdxl-base-1.0-fbadataset5e-4-lrwrmp0-ep20-withoutpadding-noflip-lora-newlabels"
model_name = (model_path).split("/")[-1]
print("Name of the model: ", model_name)

# Loading Trained LoRa weights
pipe.load_lora_weights(model_path)

/home/bsuetcueoglu/testprojectthesis/thesistest/thesisProjectTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-22 11:35:52.910900: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 11:35:52.956662: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-22 11:35:53.863748: W tensorflow/compiler/tf2tensorrt/utils/py_

Name of the model:  sdxl-base-1.0-fbadataset5e-4-lrwrmp0-ep20-withoutpadding-noflip-lora-newlabels


Generate prompt that includes build-up edge wear (BUE)
- Prompt for only BUE wear
- Random prompt including BUE wear (can contain other wear types too)

In [3]:
total_num_images = 1
tool_type = "insert"
cat = "bue"
prompt = PromptGenerator.generate_with_one_cat(tool_type, cat)

os.makedirs(f"./generated_images/{model_name}/bue_only", exist_ok=True)
for i in range(total_num_images):
    image = pipe(
        prompt=prompt,
        num_inference_steps=50,
        width=1280,
        height=1024,
        guidance_scale= 7.0 #how much to prompt effects the generated image
    ).images[0]
    
    image.save(f"./generated_images/{model_name}/bue_only/{prompt}_{i}.bmp")

100%|██████████| 50/50 [00:11<00:00,  4.33it/s]


Generating images for Groove category

In [4]:
total_num_images = 1
tool_type = "insert"
cat = "groove"
prompt = PromptGenerator.generate_with_one_cat(tool_type, cat)

os.makedirs(f"./generated_images/{model_name}/groove_only", exist_ok=True)

for i in range(total_num_images):
    image = pipe(
        prompt=prompt,
        num_inference_steps=50,
        width=1280,
        height=1024,
        guidance_scale= 7.0 #how much to prompt effects the generated image
    ).images[0]

    image.save(f"./generated_images/{model_name}/groove_only/{prompt}_{i}.bmp")

100%|██████████| 50/50 [00:11<00:00,  4.37it/s]


In [5]:
total_num_images = 1
tool_type = "insert"
categories = ["bue", "groove", "flank"]
cat = "bue"
prompt = ""
os.makedirs(f"./generated_images/{model_name}/bue", exist_ok=True)

for i in range(total_num_images):
    random_prompt = PromptGenerator.generate_random_prompt_with_a_category(tool_type=tool_type, categories=categories, fixed_cat=cat)
    image = pipe(
        prompt=random_prompt,
        num_inference_steps=50,
        width=1280,
        height=1024,
        guidance_scale= 7.0 #how much to prompt effects the generated image
    ).images[0]

    image.save(f"./generated_images/{model_name}/bue/{random_prompt}_{i}.bmp")

100%|██████████| 50/50 [00:11<00:00,  4.43it/s]


Random prompt without any wear type restrictions

In [ ]:
total_num_images = 1
tool_type = "insert"
categories = ["bue", "groove", "flank"]
prompt = ""
os.makedirs(f"./generated_images/{model_name}/random_prompt", exist_ok=True)

for i in range (total_num_images):
    random_prompt = PromptGenerator.generate_random_prompt(tool_type, categories)
    # print(prompt)
    image = pipe(
        prompt=random_prompt,
        num_inference_steps=50,
        width=1280,
        height=1024,
        guidance_scale= 7.0 #how much to prompt effects the generated image
    ).images[0]

    image.save(f"./generated_images/{model_name}/random_prompt/{random_prompt}_{i}.bmp")

100%|██████████| 50/50 [00:11<00:00,  4.40it/s]
